# **Bitcoin price prediction - Generalized Linear Regression**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: executing the chosen model, first with default values, then by choosing the best parameters by performing hyperparameter tuning with cross validation and performance evaluation.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = True # Decide whether or not to use operations that might slow down notebook execution
SPLITTING_METHOD = "walk_forward_splits"
MODEL_NAME = "GeneralizedLinearRegression"
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING: 
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets names
DATASET_TRAIN_VALID_NAME = "bitcoin_blockchain_data_15min_train_valid"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features labels
FEATURES_LABEL = "features"
TARGET_LABEL = "next-market-price"

# Features names
ALL_FEATURES_NAME = "all_features"
MOST_REL_FEATURES_NAME = "most_rel_features"
LEAST_REL_FEATURES_NAME = "least_rel_features"

# Features paths
ALL_FEATURES = FEATURES_DIR + "/" + ALL_FEATURES_NAME + ".json"
MOST_REL_FEATURES = FEATURES_DIR + "/" + MOST_REL_FEATURES_NAME + ".json"
LEAST_REL_FEATURES = FEATURES_DIR + "/" + LEAST_REL_FEATURES_NAME + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

#####################
# --- UTILITIES --- #
#####################

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + ".csv"
MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [4]:
# Suppression of warnings for better reading
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
if not LOCAL_RUNNING:
    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

# Import files

In [6]:
# Import my files
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import utilities, parameters

importlib.reload(utilities)
importlib.reload(parameters)

<module 'parameters' from 'D:\\Documents/Repository/BDC/project/utilities\\parameters.py'>

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
def dataset_info(dataset):
  # Print dataset
  dataset.show(3)

  # Get the number of rows
  num_rows = dataset.count()

  # Get the number of columns
  num_columns = len(dataset.columns)

  # Print the shape of the dataset
  print("Shape:", (num_rows, num_columns))

  # Print the schema of the dataset
  dataset.printSchema()

In [10]:
if SLOW_OPERATIONS:
  dataset_info(df)

+-------------------+---+------------------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+--------------------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          timestamp| id|      market-price|      total-bitcoins|          market-cap|        trade-volume|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|   n-transactions|estimated-transaction-volume-usd|      rate-of-change|        sma-5-days|        sma-7-days|       sma-10-days|       sma-20-days|       sma-50-days|      sma-100-days| next-market-price|
+-----

# Loading features

In [11]:
# Loading all the features
with open(ALL_FEATURES, "r") as f:
    ALL_FEATURES = json.load(f)
print(ALL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume', 'sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


In [12]:
# Loading the most relevant features
with open(MOST_REL_FEATURES, "r") as f:
    MOST_REL_FEATURES = json.load(f)
print(MOST_REL_FEATURES)

['market-price', 'market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days', 'trade-volume']


In [13]:
# Loading least relevant features
with open(LEAST_REL_FEATURES, "r") as f:
    LEAST_REL_FEATURES = json.load(f)
print(LEAST_REL_FEATURES)

['sma-100-days', 'n-unique-addresses', 'transaction-fees-usd', 'total-bitcoins', 'n-transactions-total', 'blocks-size', 'sma-50-days', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions', 'rate-of-change']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- **Default without normalization:** Make predictions using the chosen base model
- **Default with normalization:** Like the previous one but features are normalized

At this point, the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:

- **Hyperparameter tuning:** Researching the best parameters to use
- **Cross Validation:** Validate the performance of the model with the chosen parameters

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved to later make predictions on the test set.

---

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem):

- **Block time series splits:** Involves dividing the time series into blocks of equal length, and then using each block as a separate fold for cross-validation.

    ![block-splits.png](https://drive.google.com/uc?id=1SPT133HO1VdWYZZv6GeknFY3xX3T2tvL)

- **Walk forward time series splits:** Involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

    ![walk-forward-splits.png](https://drive.google.com/uc?id=1SNdq-kjbv4MXtdBj3EOJ2dmQpbbPStJi)

- **Single time series split:** Involves dividing the time series considering as validation set a narrow period of time and as train set everything that happened before this period, in such a way as to best benefit from the trend in the short term.

    ![single-split.png](https://drive.google.com/uc?id=1SODyQLolK4zn9lFGnNaqnMBZrHn3OsVn)

In [14]:
# Get splitting parameters based on the choosen splitting method
splitting_info = parameters.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Get default parameters
params = parameters.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [25], 'regParam': [0]}

In [17]:
# Make predictions by using all the features
default_res_all, default_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_all

{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 1.544627280293475, 'MSE': 2.385873435026818, 'MAE': 1.273310205472029, 'MAPE': 0.00014114690073462725, 'R2': 0.999996862414024, 'Adjusted_R2': 0.9999968599014426, 'Time': 3.3500564098358154}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 8.733650369761847, 'MSE': 76.27664878124125, 'MAE': 7.103079685648559, 'MAPE': 0.0007416044965220376, 'R2': 0.9996874026465642, 'Adjusted_R2': 0.9996871523183533, 'Time': 1.0303122997283936}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 2.8579445

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,walk_forward_splits,all_features,1,"(20000, 5000)","[25, 0]",1.544627,2.385873,1.273310,0.000141,0.999997,0.999997,3.350056
1,GeneralizedLinearRegression,default,walk_forward_splits,all_features,2,"(20000, 5000)","[25, 0]",8.733650,76.276649,7.103080,0.000742,0.999687,0.999687,1.030312
2,GeneralizedLinearRegression,default,walk_forward_splits,all_features,3,"(20000, 5000)","[25, 0]",2.857945,8.167847,2.461786,0.000223,0.999976,0.999976,0.992509
3,GeneralizedLinearRegression,default,walk_forward_splits,all_features,4,"(20000, 5000)","[25, 0]",38.353223,1470.969715,32.770936,0.002507,0.999486,0.999486,1.173375
4,GeneralizedLinearRegression,default,walk_forward_splits,all_features,5,"(20000, 5000)","[25, 0]",4.295739,18.453374,2.983149,0.000137,0.999999,0.999999,0.944168
5,GeneralizedLinearRegression,default,walk_forward_splits,all_features,6,"(20000, 5000)","[25, 0]",60.669631,3680.804079,46.518445,0.001165,0.999933,0.999933,0.848700
6,GeneralizedLinearRegression,default,walk_forward_splits,all_features,7,"(20000, 5000)","[25, 0]",131.365094,17256.787947,117.719095,0.002057,0.999197,0.999196,0.822920
7,GeneralizedLinearRegression,default,walk_forward_splits,all_features,8,"(20000, 5000)","[25, 0]",126.757007,16067.338796,115.552274,0.002756,0.999789,0.999789,0.923887
8,GeneralizedLinearRegression,default,walk_forward_splits,all_features,9,"(20000, 5000)","[25, 0]",37.718249,1422.666275,31.325168,0.000909,0.999805,0.999805,0.810666
9,GeneralizedLinearRegression,default,walk_forward_splits,all_features,10,"(20000, 5000)","[25, 0]",4.517135,20.404508,3.753351,0.000080,0.999998,0.999998,0.903859


In [18]:
# Make predictions by using the most relevant features
default_res_most_rel, default_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_most_rel

{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 2.7035860182874094, 'MSE': 7.309377358279168, 'MAE': 1.978041358383934, 'MAPE': 0.00022433394842406065, 'R2': 0.9999903876712167, 'Adjusted_R2': 0.9999903799736561, 'Time': 0.42952728271484375}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 2.9811151144183046, 'MSE': 8.887047325413262, 'MAE': 2.4750849264021966, 'MAPE': 0.0002581978181830562, 'R2': 0.9999635790570486, 'Adjusted_R2': 0.9999635498911283, 'Time': 0.37917399406433105}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [25, 

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[25, 0]",2.703586,7.309377,1.978041,0.000224,0.999990,0.999990,0.429527
1,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[25, 0]",2.981115,8.887047,2.475085,0.000258,0.999964,0.999964,0.379174
2,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[25, 0]",3.175634,10.084649,2.240039,0.000199,0.999970,0.999970,0.388538
3,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[25, 0]",4.508862,20.329841,3.493285,0.000272,0.999993,0.999993,0.460680
4,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[25, 0]",7.115173,50.625691,5.281364,0.000253,0.999997,0.999997,0.389436
5,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[25, 0]",18.688065,349.243771,14.642597,0.000369,0.999994,0.999994,0.386668
6,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[25, 0]",20.640611,426.034839,16.787039,0.000305,0.999980,0.999980,0.393255
7,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[25, 0]",24.218816,586.551070,17.598453,0.000385,0.999992,0.999992,0.427280
8,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[25, 0]",17.236187,297.086140,14.823408,0.000434,0.999959,0.999959,0.455679
9,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[25, 0]",17.137014,293.677260,12.950529,0.000281,0.999968,0.999968,0.404729


In [19]:
# Make predictions by using the least relevant features
default_res_least_rel, default_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_res_least_rel

{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 4754.974644396952, 'MSE': 22609783.868857913, 'MAE': 3192.911610339546, 'MAPE': 0.34431224087814793, 'R2': -28.733404859884462, 'Adjusted_R2': -28.757215394306794, 'Time': 0.5282502174377441}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 897.9342496159495, 'MSE': 806285.9166333583, 'MAE': 784.3428907393667, 'MAPE': 0.08331318160350913, 'R2': -2.3043250808648565, 'Adjusted_R2': -2.3069711870357192, 'Time': 0.8611128330230713}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0]

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[25, 0]",4754.974644,2.260978e+07,3192.911610,0.344312,-28.733405,-28.757215,0.528250
1,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[25, 0]",897.934250,8.062859e+05,784.342891,0.083313,-2.304325,-2.306971,0.861113
2,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[25, 0]",1547.495535,2.394742e+06,1342.578320,0.116819,-6.062510,-6.068166,0.570640
3,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[25, 0]",1452.227756,2.108965e+06,1146.471861,0.092394,0.263278,0.262688,0.484572
4,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[25, 0]",4348.085446,1.890585e+07,3883.268528,0.178752,-0.180427,-0.181373,0.751659
5,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[25, 0]",7678.317162,5.895655e+07,6758.999458,0.172630,-0.067856,-0.068711,0.486738
6,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[25, 0]",15265.911389,2.330481e+08,12180.818999,0.211317,-9.844444,-9.853128,0.484966
7,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[25, 0]",17363.034617,3.014750e+08,16133.424111,0.377951,-2.951117,-2.954281,0.475173
8,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[25, 0]",6643.073422,4.413042e+07,5821.663565,0.165155,-5.053438,-5.058286,0.551260
9,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[25, 0]",3675.196850,1.350707e+07,3246.046446,0.069499,-0.481913,-0.483099,0.759178


### With normalization

In [20]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [21]:
# Make predictions by using all the features
default_norm_res_all, default_norm_pred_all = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, ALL_FEATURES, ALL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_all

{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 299.5183036611932, 'MSE': 89711.21422807875, 'MAE': 230.59807561212094, 'MAPE': 0.0247420935495992, 'R2': 0.8820236465511144, 'Adjusted_R2': 0.881929170992797, 'Time': 0.556490421295166}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 181.52287817012402, 'MSE': 32950.55529916569, 'MAE': 155.1784703544043, 'MAPE': 0.016262852373327533, 'R2': 0.8649618651928306, 'Adjusted_R2': 0.8648537265463383, 'Time': 0.6628429889678955}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE'

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,1,"(20000, 5000)","[25, 0]",299.518304,8.971121e+04,230.598076,0.024742,0.882024,0.881929,0.556490
1,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,2,"(20000, 5000)","[25, 0]",181.522878,3.295056e+04,155.178470,0.016263,0.864962,0.864854,0.662843
2,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,3,"(20000, 5000)","[25, 0]",244.297704,5.968137e+04,222.725487,0.019680,0.823989,0.823848,0.607069
3,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,4,"(20000, 5000)","[25, 0]",1048.578227,1.099516e+06,833.248148,0.062857,0.615907,0.615600,0.641821
4,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,5,"(20000, 5000)","[25, 0]",2789.448428,7.781023e+06,1874.231086,0.077140,0.514175,0.513786,0.548355
5,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,6,"(20000, 5000)","[25, 0]",2267.060731,5.139564e+06,1864.413079,0.042806,0.906909,0.906835,0.779735
6,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,7,"(20000, 5000)","[25, 0]",1754.289928,3.077533e+06,1634.029552,0.028995,0.856793,0.856678,0.618499
7,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,8,"(20000, 5000)","[25, 0]",14651.669469,2.146714e+08,12395.190374,0.299163,-1.813474,-1.815727,0.610258
8,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,9,"(20000, 5000)","[25, 0]",10287.500767,1.058327e+08,8541.425107,0.250873,-13.517231,-13.528856,0.731873
9,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,10,"(20000, 5000)","[25, 0]",5046.131038,2.546344e+07,4361.225320,0.092077,-1.793692,-1.795929,0.660788


In [22]:
# Make predictions by using the most relevant features
default_norm_res_most_rel, default_norm_pred_most_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, MOST_REL_FEATURES, MOST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_most_rel

{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 179.4242395225593, 'MSE': 32193.05772824873, 'MAE': 149.97306037275382, 'MAPE': 0.01711879030498661, 'R2': 0.9576639376712448, 'Adjusted_R2': 0.9576300349186292, 'Time': 0.3440077304840088}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 710.776803948509, 'MSE': 505203.66503125726, 'MAE': 664.1594212257036, 'MAPE': 0.0708664597878427, 'R2': -1.0704282523971407, 'Adjusted_R2': -1.0720862529996609, 'Time': 0.365281343460083}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'most_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Parameters':

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,1,"(20000, 5000)","[25, 0]",179.424240,3.219306e+04,149.973060,0.017119,0.957664,0.957630,0.344008
1,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,2,"(20000, 5000)","[25, 0]",710.776804,5.052037e+05,664.159421,0.070866,-1.070428,-1.072086,0.365281
2,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,3,"(20000, 5000)","[25, 0]",638.856889,4.081381e+05,584.930181,0.051348,-0.203670,-0.204634,0.366042
3,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,4,"(20000, 5000)","[25, 0]",1495.049703,2.235174e+06,1058.281627,0.076646,0.219190,0.218565,0.361727
4,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,5,"(20000, 5000)","[25, 0]",6024.816108,3.629841e+07,5188.731281,0.232483,-1.266369,-1.268184,0.387170
5,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,6,"(20000, 5000)","[25, 0]",13298.057034,1.768383e+08,11460.739599,0.266408,-2.203000,-2.205565,0.404283
6,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,7,"(20000, 5000)","[25, 0]",8877.709537,7.881373e+07,8033.298215,0.139733,-2.667446,-2.670383,0.399042
7,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,8,"(20000, 5000)","[25, 0]",23602.484662,5.570773e+08,19833.484983,0.490896,-6.301029,-6.306875,0.396863
8,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,9,"(20000, 5000)","[25, 0]",27675.720998,7.659455e+08,23804.234639,0.698698,-104.065930,-104.150067,0.448244
9,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,10,"(20000, 5000)","[25, 0]",45235.219560,2.046225e+09,39721.666494,0.846130,-223.499217,-223.678996,0.451101


In [23]:
# Make predictions by using the least relevant features
default_norm_res_least_rel, default_norm_pred_least_rel = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, LEAST_REL_FEATURES, LEAST_REL_FEATURES_NAME, FEATURES_LABEL, TARGET_LABEL)
default_norm_res_least_rel

{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 1756.4831869776199, 'MSE': 3085233.186135056, 'MAE': 1402.4758833445821, 'MAPE': 0.15411337846067177, 'R2': -3.0572916549130493, 'Adjusted_R2': -3.0605407373193865, 'Time': 0.4209010601043701}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [25, 0], 'RMSE': 1867.17753237837, 'MSE': 3486351.937418579, 'MAE': 1661.2637770571616, 'MAPE': 0.17498369706848763, 'R2': -13.287785399546415, 'Adjusted_R2': -13.299227069536043, 'Time': 0.44104623794555664}
{'Model': 'GeneralizedLinearRegression', 'Type': 'default_norm', 'Splitting': 'walk_forward_splits', 'Features': 'least_rel_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'Par

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,1,"(20000, 5000)","[25, 0]",1756.483187,3.085233e+06,1402.475883,0.154113,-3.057292,-3.060541,0.420901
1,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,2,"(20000, 5000)","[25, 0]",1867.177532,3.486352e+06,1661.263777,0.174984,-13.287785,-13.299227,0.441046
2,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,3,"(20000, 5000)","[25, 0]",1734.491688,3.008461e+06,1586.107059,0.139288,-7.872474,-7.879579,0.402218
3,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,4,"(20000, 5000)","[25, 0]",1171.285013,1.371909e+06,888.976633,0.071319,0.520753,0.520369,0.410091
4,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,5,"(20000, 5000)","[25, 0]",7176.898909,5.150788e+07,6417.948973,0.292410,-2.216005,-2.218581,0.450892
5,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,6,"(20000, 5000)","[25, 0]",5506.743922,3.032423e+07,4660.171576,0.124021,0.450750,0.450310,0.682808
6,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,7,"(20000, 5000)","[25, 0]",12115.514130,1.467857e+08,9920.992361,0.172725,-5.830390,-5.835860,0.495586
7,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,8,"(20000, 5000)","[25, 0]",21992.792904,4.836829e+08,20773.156378,0.486942,-5.339126,-5.344202,0.568393
8,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,9,"(20000, 5000)","[25, 0]",13792.022199,1.902199e+08,13217.934303,0.380769,-25.092754,-25.113649,0.595652
9,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,10,"(20000, 5000)","[25, 0]",14472.507883,2.094535e+08,13998.992333,0.298235,-21.979947,-21.998349,0.490576


In [24]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [25]:
# Define the results to show
default_comparison_lst = [default_res_all, default_res_most_rel, default_res_least_rel, default_norm_res_all, default_norm_res_most_rel, default_norm_res_least_rel]

# Show the comparison table
default_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,walk_forward_splits,all_features,"[25, 0]",22.956868,1.872029e+03,19.655343,0.000621,0.999838,0.999838,0.932029
0,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,"[25, 0]",11.143603,1.648257e+02,8.621713,0.000276,0.999976,0.999976,0.419835
0,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,"[25, 0]",12250.863053,4.187280e+08,9903.491425,0.287631,-25.769681,-25.791118,0.571865
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,"[25, 0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.647148
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,"[25, 0]",9119.947927,1.908665e+08,7993.093986,0.240692,-28.992510,-29.016528,0.418097
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,"[25, 0]",9279.897142,1.434210e+08,8258.956310,0.263110,-23.694272,-23.714047,0.505194


In [26]:
# Save the best default model results and predicitons
best_default_results = default_norm_res_all
best_default_predictions = default_norm_pred_all

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

### Hyperparameter tuning

In [27]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"
CHOSEN_FEATURES = ALL_FEATURES
CHOSEN_FEATURES_LABEL = ALL_FEATURES_NAME
FEATURES_NORMALIZATION = True

In [28]:
# Get model grid parameters
params = parameters.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [5, 10, 50, 80],
 'regParam': [0, 0.1, 0.2],
 'family': ['gaussian', 'gamma'],
 'link': ['log', 'identity', 'inverse']}

In [29]:
# Perform hyperparameter tuning
hyp_res = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
hyp_res

{'Model': 'GeneralizedLinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.2, 'gaussian', 'log'], 'RMSE': 212.2260711009864, 'MSE': 45039.90525496093, 'MAE': 160.99767762123412, 'MAPE': 0.017303849024711458, 'R2': 0.9407694586748724, 'Adjusted_R2': 0.9407220268099473, 'Time': 0.13457775115966797}
{'Model': 'GeneralizedLinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0.2, 'gaussian', 'log'], 'RMSE': 70.05198035283917, 'MSE': 4907.279951354565, 'MAE': 60.99220203438704, 'MAPE': 0.0064824974345667795, 'R2': 0.9798889601224925, 'Adjusted_R2': 0.9798728551856537, 'Time': 0.11057043075561523}
{'Model': 'GeneralizedLinearRegression', 'Type': 'hyp_tuning', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (200

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,1,"(20000, 5000)","[5, 0.2, gaussian, log]",212.226071,4.503991e+04,160.997678,0.017304,0.940769,0.940722,0.134578
1,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,2,"(20000, 5000)","[5, 0.2, gaussian, log]",70.051980,4.907280e+03,60.992202,0.006482,0.979889,0.979873,0.110570
2,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,3,"(20000, 5000)","[5, 0.1, gaussian, log]",220.925360,4.880801e+04,198.952304,0.017527,0.856057,0.855941,0.097581
3,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,4,"(20000, 5000)","[5, 0.1, gaussian, log]",207.994925,4.326189e+04,159.381291,0.011987,0.984887,0.984875,0.109042
4,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,5,"(20000, 5000)","[5, 0.2, gaussian, log]",2634.327346,6.939681e+06,1722.797342,0.070273,0.566706,0.566359,0.099779
5,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,6,"(20000, 5000)","[5, 0.1, gaussian, log]",1480.474626,2.191805e+06,1155.510533,0.026407,0.960301,0.960269,0.109764
6,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,7,"(20000, 5000)","[5, 0, gaussian, log]",1754.289928,3.077533e+06,1634.029552,0.028995,0.856793,0.856678,0.586014
7,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,8,"(20000, 5000)","[5, 0.2, gaussian, log]",10272.010183,1.055142e+08,8748.618595,0.209880,-0.382864,-0.383971,0.095563
8,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,9,"(20000, 5000)","[5, 0.1, gaussian, log]",8823.642049,7.785666e+07,7238.947842,0.212859,-9.679718,-9.688270,0.100335
9,GeneralizedLinearRegression,hyp_tuning,walk_forward_splits,all_features,10,"(20000, 5000)","[5, 0.1, gaussian, log]",1665.990431,2.775524e+06,1620.720899,0.035122,0.695487,0.695243,0.113937


---
To select the best parameters to be used in the final model I assign a score to each value in the "Parameters" column based on the following criteria:
* Calculate the frequency of each unique value in the "Parameters" column.
* Normalize the frequencies to a scale of 0 to 1, where 1 represents the highest frequency.
* Calculate the split weight for each value in the "Parameters" column, where a higher split number corresponds to a higher weight.
* Normalize the split weights to a scale of 0 to 1, where 1 represents the highest split weight.
* Calculate the RMSE weight for each value in the "Parameters" column, where a lower RMSE value corresponds to a higher weight.
* Normalize the RMSE weights to a scale of 0 to 1, where 1 represents the highest RMSE weight.

Then calculate the overall score for each value in the "Parameters" column by combining the normalized frequency, split weight, and RMSE weight and take into consideration the parameters that have the highest score.

In [30]:
# Show parameters score
grouped_scores, best_params = parameters.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(5, 0, gaussian, log)",0.719697,0.916083,0.666667,0.439535
"(5, 0.1, gaussian, log)",0.500000,0.827359,1.000000,0.413679
"(5, 0.2, gaussian, log)",0.383117,0.738627,0.777778,0.220096


In [31]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (5, 0, 'gaussian', 'log')


### Cross validation

In [32]:
MODEL_TYPE = "cross_val"

In [33]:
# Get tuned parameters
params = parameters.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [5], 'regParam': [0], 'family': ['gaussian'], 'link': ['log']}

In [34]:
# Perform cross validation
cv_res, cv_pred = utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL)
cv_res

{'Model': 'GeneralizedLinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 1, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0, 'gaussian', 'log'], 'RMSE': 299.5183036611932, 'MSE': 89711.21422807875, 'MAE': 230.59807561212094, 'MAPE': 0.0247420935495992, 'R2': 0.8820236465511144, 'Adjusted_R2': 0.881929170992797, 'Time': 0.572324275970459}
{'Model': 'GeneralizedLinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 2, 'Train&Validation': (20000, 5000), 'Parameters': [5, 0, 'gaussian', 'log'], 'RMSE': 181.52287817012402, 'MSE': 32950.55529916569, 'MAE': 155.1784703544043, 'MAPE': 0.016262852373327533, 'R2': 0.8649618651928306, 'Adjusted_R2': 0.8648537265463383, 'Time': 0.5711205005645752}
{'Model': 'GeneralizedLinearRegression', 'Type': 'cross_val', 'Splitting': 'walk_forward_splits', 'Features': 'all_features', 'Splits': 3, 'Train&Validation': (20000, 5000), 'P

,Model,Type,Splitting,Features,Splits,Train&Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,1,"(20000, 5000)","[5, 0, gaussian, log]",299.518304,8.971121e+04,230.598076,0.024742,0.882024,0.881929,0.572324
1,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,2,"(20000, 5000)","[5, 0, gaussian, log]",181.522878,3.295056e+04,155.178470,0.016263,0.864962,0.864854,0.571121
2,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,3,"(20000, 5000)","[5, 0, gaussian, log]",244.297704,5.968137e+04,222.725487,0.019680,0.823989,0.823848,0.491914
3,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,4,"(20000, 5000)","[5, 0, gaussian, log]",1048.578227,1.099516e+06,833.248148,0.062857,0.615907,0.615600,0.554744
4,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,5,"(20000, 5000)","[5, 0, gaussian, log]",2789.448428,7.781023e+06,1874.231086,0.077140,0.514175,0.513786,0.421971
5,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,6,"(20000, 5000)","[5, 0, gaussian, log]",2267.060731,5.139564e+06,1864.413079,0.042806,0.906909,0.906835,0.525071
6,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,7,"(20000, 5000)","[5, 0, gaussian, log]",1754.289928,3.077533e+06,1634.029552,0.028995,0.856793,0.856678,0.520024
7,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,8,"(20000, 5000)","[5, 0, gaussian, log]",14651.669469,2.146714e+08,12395.190374,0.299163,-1.813474,-1.815727,0.576366
8,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,9,"(20000, 5000)","[5, 0, gaussian, log]",10287.500767,1.058327e+08,8541.425107,0.250873,-13.517231,-13.528856,0.551138
9,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,10,"(20000, 5000)","[5, 0, gaussian, log]",5046.131038,2.546344e+07,4361.225320,0.092077,-1.793692,-1.795929,0.543298


In [35]:
# Define the results to show
tuned_comparison_lst = [cv_res]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0, gaussian, log]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.50625,0.542587


# Comparison table
Visualization of model performance at various stages of train / validation

In [36]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default,walk_forward_splits,all_features,"[25, 0]",22.956868,1.872029e+03,19.655343,0.000621,0.999838,0.999838,0.932029
1,GeneralizedLinearRegression,default,walk_forward_splits,most_rel_features,"[25, 0]",11.143603,1.648257e+02,8.621713,0.000276,0.999976,0.999976,0.419835
2,GeneralizedLinearRegression,default,walk_forward_splits,least_rel_features,"[25, 0]",12250.863053,4.187280e+08,9903.491425,0.287631,-25.769681,-25.791118,0.571865
3,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,"[25, 0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.647148
4,GeneralizedLinearRegression,default_norm,walk_forward_splits,most_rel_features,"[25, 0]",9119.947927,1.908665e+08,7993.093986,0.240692,-28.992510,-29.016528,0.418097
5,GeneralizedLinearRegression,default_norm,walk_forward_splits,least_rel_features,"[25, 0]",9279.897142,1.434210e+08,8258.956310,0.263110,-23.694272,-23.714047,0.505194
6,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0, gaussian, log]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.506250,0.542587


# Model accuracy

Since predicting the price accurately is very difficult let's se how good the model is at predicting whether the price will go up or down. 

For each row in our predictions let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
We compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that we count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally we compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [37]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_pred)

# Compute model accuracy
default_accuracy = utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = utilities.model_accuracy(validated_pred_spark)

# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (validated)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (validated): {validated_accuracy:.2f}%")

Percentage of correct predictions for GeneralizedLinearRegression with all_features and walk_forward_splits (default): 49.33%
Percentage of correct predictions for GeneralizedLinearRegression with all_features and walk_forward_splits (validated): 49.33%


# Saving final results


In [38]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_res]
default_tuned_results_df = pd.concat([utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GeneralizedLinearRegression,default_norm,walk_forward_splits,all_features,"[25, 0]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.50625,0.647148
0,GeneralizedLinearRegression,cross_val,walk_forward_splits,all_features,"[5, 0, gaussian, log]",2547.856094,1.836015e+07,2145.693637,0.062408,-0.505045,-0.50625,0.542587


In [39]:
# Saving default and tuned results
default_tuned_results_df.to_csv(MODEL_RESULTS, index=False)

In [40]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)